# SQS - Simple Queue Service Notes

Notes on the AWS SQS service illustrated with code examples in Python using the [AWS Python SDK](https://boto3.readthedocs.io/en/latest/reference/services/index.html)

This particular notebook shows access at the raw service level - sqsResource notebook shows the resource oriented API

## The Basics

### Create a queue

The following shows how to create queue with just the queue name. This will create a queue of the
[standard type](http://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/Welcome.html#sqs-queue-types). To create a fifo queue, we would use a name with a `.fifo` suffix.

In [ ]:
import boto3

client = boto3.client('sqs')
create_response = client.create_queue(
        QueueName = 'sampleQ'
    )
print(create_response)

### Listing Queues

In [ ]:
# List all queues
queueList = client.list_queues()
print queueList

# List queues that begin with 'sample'
queueList = client.list_queues( QueueNamePrefix = 'sample')
print '\n'
print queueList

### Send a Message to the Queue

In [ ]:
response = client.send_message(
    QueueUrl = create_response['QueueUrl'],
    MessageBody='Hello there',
    DelaySeconds=0,
    MessageAttributes={
        'attributeOne': {
            'StringValue':'attribute one value',
            'DataType':'String'
        }
    },
    MessageDeduplicationId='1',
    MessageGroupId='1'
)

print response

### Receieve a Message From a Queue

This shows just a basic receive. To understand how what values to use for max number of messages, visibility, and
wait timeout, you need to think about the system you are designing - what sort of concurrency will apply to pulling
messages from the queue, how long will it take to process the message, and so on.

In [ ]:
response = client.receive_message(
    QueueUrl = create_response['QueueUrl'],
    AttributeNames=[
        'All'
    ],
    MaxNumberOfMessages=1,
    VisibilityTimeout=5,
    WaitTimeSeconds=5
)

print response

receipt_handle = response['Messages'][0]['ReceiptHandle']

### Delete a Message

In [ ]:
response = client.delete_message(
    QueueUrl=create_response['QueueUrl'],
    ReceiptHandle=receipt_handle
)

print response

### Delete a Queue

In [ ]:
response = client.delete_queue(
    QueueUrl = create_response['QueueUrl']
)

print(response)

## Using a Dead Letter Queue

### Create the Queues

In [ ]:
import json

## Create the dead letter queue
dlq_resp = client.create_queue(
        QueueName = 'sampleDeadLetterQueue'
    )
print(dlq_resp)

response = client.get_queue_attributes(
    QueueUrl=dlq_resp['QueueUrl'],
    AttributeNames=[
        'QueueArn'
    ]
)

dql_arn = response['Attributes']['QueueArn']

In [ ]:
## Create the redrive policy
redrivePolicy = {}
redrivePolicy['maxReceiveCount'] = 2
redrivePolicy['deadLetterTargetArn'] = dql_arn
print json.dumps(redrivePolicy)


In [ ]:
## Create the sample queue with the DLQ
sample_q_response = client.create_queue(
    QueueName = 'sampleQwithDLQ',
     Attributes={
        'RedrivePolicy': json.dumps(redrivePolicy)
    }
)

In [ ]:
## Read the policy via get_queue_attributes
response = client.get_queue_attributes(
    QueueUrl=sample_q_response['QueueUrl'],
    AttributeNames=[
        'RedrivePolicy'
    ]
)

print response

In [ ]:
## List source queue for the DLQ

response = client.list_dead_letter_source_queues(
    QueueUrl = dlq_resp['QueueUrl']
)

print response

In [ ]:
## Enqueue a message
response = client.send_message(
    QueueUrl = sample_q_response['QueueUrl'],
    MessageBody='Hello there',
    DelaySeconds=0,
    MessageAttributes={
        'attributeOne': {
            'StringValue':'attribute one value',
            'DataType':'String'
        }
    },
    MessageDeduplicationId='1',
    MessageGroupId='1'
)

In [ ]:
import time

## Receive the message twice
for i in range(0, 3):
    print 'receive ', i
    
    response = client.receive_message(
        QueueUrl = sample_q_response['QueueUrl'],
        AttributeNames=[
            'All'
        ],
        MaxNumberOfMessages=1,
        VisibilityTimeout=1,
        WaitTimeSeconds=1
    )
    
    print response
    
    time.sleep(2)
    

In [ ]:
## Receive from DLQ
response = client.receive_message(
    QueueUrl = dlq_resp['QueueUrl'],
    AttributeNames=[
        'All'
    ],
    MaxNumberOfMessages=1,
    VisibilityTimeout=1,
    WaitTimeSeconds=1
)

print response

### Clean Up

In [ ]:
response = client.delete_queue(
    QueueUrl = dlq_resp['QueueUrl']
)

print(response)